In [4]:
import simpy

In [3]:
!python -V

Python 3.5.2 :: Anaconda custom (x86_64)


In [57]:
>>> class Car(object):
...     def __init__(self, env):
...         self.env = env
...         self.action = env.process(self.run())
...
...     def run(self):
...         while True:
...             print('Start parking and charging at %d' % self.env.now)
...             charge_duration = 5
...             # We may get interrupted while charging the battery
...             try:
...                 yield self.env.process(self.charge(charge_duration))
...             except simpy.Interrupt:
...                 # When we received an interrupt, we stop charging and
...                 # switch to the "driving" state
...                 print('Was interrupted. Hope, the battery is full enough ...')
...
...             print('Start driving at %d' % self.env.now)
...             trip_duration = 2
...             yield self.env.timeout(trip_duration)
...
...     def charge(self, duration):
...         yield self.env.timeout(duration)

>>> def driver(env, car):
...     yield env.timeout(3)
...     car.action.interrupt()


>>> def car(env, name, bcs, driving_time, charge_duration):
        # Simulate driving to the BCS
        yield env.timeout(driving_time)
        # Request one of its charging spots
        print('%s arriving at %d' % (name, env.now))
        number_of_cars_charging = 0
        with bcs.request() as req:
            yield req
            
            # Charge the battery
            print('%s starting to charge at %s' % (name, env.now))
            number_of_cars_charging+=1
            print()
            print('There are {} cars charging start'.format(number_of_cars_charging))
            yield env.timeout(charge_duration)
            print('%s leaving the bcs at %s' % (name, env.now))
            number_of_cars_charging -=1
            print('There are {} cars charging left'.format(number_of_cars_charging))



In [58]:
>>> env = simpy.Environment()
>>> bcs = simpy.Resource(env, capacity=2)

for i in range(4):
    print(i)
    env.process(car(env, 'Car %d' % i, bcs, i*2, 5))

0
1
2
3


In [59]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0

There are 1 cars charging start
Car 1 arriving at 2
Car 1 starting to charge at 2

There are 1 cars charging start
Car 2 arriving at 4
Car 0 leaving the bcs at 5
There are 0 cars charging left
Car 2 starting to charge at 5

There are 1 cars charging start
Car 3 arriving at 6
Car 1 leaving the bcs at 7
There are 0 cars charging left
Car 3 starting to charge at 7

There are 1 cars charging start
Car 2 leaving the bcs at 10
There are 0 cars charging left
Car 3 leaving the bcs at 12
There are 0 cars charging left


In [112]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""
import random

import simpy




def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    global total_card_holders ### should set this as a class variable in the future
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive
        
        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            total_card_holders += counter.count
            
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))
            ### assign people who reach the counter a credit card for 100 time steps
            number_of_card_holders = simpy.Resource(env, capacity=1000)
            yield env.process(purchase_card(env,name,1,number_of_card_holders))
            #purchase_card(env,name,1)
            
        
        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))

    
    
def purchase_card(env, n, number,num_card_holders):
    """Generate 'card purchases' for people who reached the bank teller"""
    global total_card_holders
    purchase_card_time = env.now
    with num_card_holders.request() as req:
        
    #card_purchaser = ('Card-Holder%02d' %  total_card_holders, card_lifetime=100.0)
  
        yield env.timeout(1) # no delay
        print( " {} got a credit card at time {}".format(n,purchase_card_time,purchase_card_time))
        



In [113]:
# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()



RANDOM_SEED = 42
NEW_CUSTOMERS = 10  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience

# Start processes and run
counter = simpy.Resource(env, capacity=1)
total_card_holders = 0
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()
print('Total card holdres finished {}'.format(total_card_holders ))

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
 Customer00 got a credit card at time 3.859488768899587
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: RENEGED after  1.174
23.7507 Customer01: Finished
 Customer01 got a credit card at time 23.750678978331123
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
37.9599 Customer03: Finished
 Customer03 got a credit card at time 37.95994607648084
40.4798 Customer04: Here I am
40.4798 Customer04: Waited  0.000
43.1401 Customer04: Finished
 Customer04 got a credit card at time 43.14010734889202
47.5190 Customer05: Here I am
47.5190 Customer05: Waited  0.000
50.5073 Customer05: Finished
 Customer05 got a credit card at time 50.50727962519127
58.0139 Customer06: Here I am
58.0139 Customer06: Waited  0.000
58.0921 Customer06: Finished
 Customer06 got a credit card at time 58.092122232466316
66.9120 Customer07: H

In [95]:
>>> def sub(env):
...     yield env.timeout(1)
...     return 23
...
>>> def parent(env):
...     ret = yield env.process(sub(env))
...     return ret
...
>>> env.run(env.process(parent(env)))


23